<a href="https://colab.research.google.com/github/EyeOfHorus396/Cohere-Semantic-Search/blob/main/Cohere_semantic_search_trec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cohere pinecone-client datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Create Embeddings

In [3]:
import cohere

co = cohere.Client(COHERE_KEY)

##Import Local Data

In [4]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##CD to MyDrive[/content/drive')]

In [5]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


#Importing Pandas to see the data headers

In [10]:
import pandas as pd

#df = pd.read_csv('Flagstar.csv')
df = pd.read_csv('Flagstar_UWConv.txt', header=None, names=['text'])
df.head()


,text
0,Conventional Underwriting Guidelines
1,GENERAL
2,These guidelines describe our underwriting req...
3,mortgages and can be superseded by changes mad...
4,Mortgage Association (Fannie Mae) and the Fede...


#Comparing the data to the one from HuggingFace

In [57]:
from datasets import load_dataset
import pandas as pd

test_trec = load_dataset('trec', split='train[:100]')
dataview = pd.DataFrame(test_trec)
dataview.head()

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


,text,coarse_label,fine_label
0,How did serfdom develop in and then leave Russ...,2,26
1,What films featured the character Popeye Doyle ?,1,5
2,How can I find a list of celebrities ' real na...,2,26
3,What fowl grabs the spotlight after the Chines...,1,2
4,What is the full form of .com ?,0,1


# Convert to DataFrame 
Changing the csv file to a dictionary format similar to a HuggingFace dataset

In [11]:
import pandas as pd


# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('Flagstar.csv')

# Convert the DataFrame to a dictionary in a format similar to a Hugging Face dataset
dataset_dict = {}

for col in df.columns:
    dataset_dict[col] = df[col].tolist()

# Print the first 10 rows of the dataset dictionary
for i in range(10):
    print(f"Row {i}: {dataset_dict['text'][i]}, {dataset_dict['text'][i]}")

Row 0: Conventional Underwriting Guidelines, Conventional Underwriting Guidelines
Row 1: GENERAL, GENERAL
Row 2: These guidelines describe our underwriting requirements for one-to-four family conforming conventional, These guidelines describe our underwriting requirements for one-to-four family conforming conventional
Row 3: mortgages and can be superseded by changes made by secondary market investors, mortgages and can be superseded by changes made by secondary market investors
Row 4: Mortgage Association (Fannie Mae) and the Federal Home Loan Mortgage Corporation (Freddie Mac) and is, Mortgage Association (Fannie Mae) and the Federal Home Loan Mortgage Corporation (Freddie Mac) and is
Row 5: not intended to replace Fannie Mae or Freddie Mac Guidelines. The guidelines are designed to establish and, not intended to replace Fannie Mae or Freddie Mac Guidelines. The guidelines are designed to establish and
Row 6: implement sound underwriting criteria, implement sound underwriting criteri

Or converting a text file to HuggingFace dataset version (use one or the other, dont run both cells)

Converting a text version instead of a csv (either convert the csv or text, not both)

In [17]:
import pandas as pd
import json

# Read in the text file as a DataFrame
df = pd.read_csv('Flagstar_UWConv.txt', sep='\t', header=None, names=['text'])

# Create an empty dictionary to store the dataset
dataset_dict = {'train': None}

# Append the DataFrame to the dataset dictionary
dataset_dict['train'] = df

# Convert the DataFrame to a dictionary
dataset_dict = df.to_dict('list')

dumping the text version into json

In [18]:
# Save the dataset dictionary to a JSON file
with open('temp_dataset.json', 'w') as f:
    json.dump(dataset_dict, f)

In [19]:
# Encode the text as UTF-8 and decode as Unicode escape
for i in range(len(dataset_dict['text'])):
    dataset_dict['text'][i] = dataset_dict['text'][i].encode('utf-8').decode('unicode_escape')

In [60]:
import json

# Save the dataset dictionary to a JSON file
with open('flagstar_dataset.json', 'w') as f:
    json.dump(dataset_dict, f)

In [20]:
from datasets import load_dataset
# Load the dataset using load_dataset()
trec = load_dataset('json', data_files={'train': 'temp_dataset.json'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b3f8fb9abd6cc75b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
# Print the dataset
print(trec['train']['text'][0])

['Conventional Underwriting Guidelines', 'GENERAL', 'These guidelines describe our underwriting requirements for one-to-four family conforming conventional', 'mortgages and can be superseded by changes made by secondary market investors, Federal National', 'Mortgage Association (Fannie Mae) and the Federal Home Loan Mortgage Corporation (Freddie Mac) and is', 'not intended to replace Fannie Mae or Freddie Mac Guidelines. The guidelines are designed to establish and', 'implement sound underwriting criteria, as well as to serve as a reference tool in tandem with the product', 'descriptions.', 'Guideline references to Fannie Mae or Freddie Mac specific lending requirements must contain the', 'corresponding AUS response. If following Fannie Mae guidelines, an acceptable DU response is required. If', 'following Freddie Mac guidelines, an acceptable LPA response is required.', 'Prior to consummation, the underwriter must make a reasonable and good faith determination of the', 'consumer’s abi

In [30]:
# trec["train"]["text"][0]

#Passing data to create embeddings in Cohere.

In [23]:
embeds = co.embed(
    texts=trec["train"]["text"][0],
    model='large',
    truncate='LEFT'
).embeddings

Check the dimensionality of the returned vectors, to be used when initializing our Pinecone index.

In [24]:
import numpy as np

shape = np.array(embeds).shape
shape

(7492, 4096)

Here we can see the `4096` embedding dimensionality produced by Cohere's small model, and the `7492` samples we built embeddings for.

## Storing the Embeddings

Now that we have our embeddings we can move on to indexing them in the Pinecone vector database. Again, this is very simple, we just initialize our connection to Pinecone and then create a new index for storing the embeddings, making sure to specify that we would like to use the cosine similarity metric to align with Cohere's embeddings.

In [25]:
import pinecone

pinecone.init(
    PINECONE_KEY,
    environment="us-east1-gcp"  # find next to API key in console
)

index_name = 'cohere-pinecone-trec'

# if the index does not exist, we create it
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=shape[1],
        metric='cosine'
    )

# connect to index
index = pinecone.Index(index_name)

Now we can begin populating the index with our embeddings. Pinecone expects us to provide a list of tuples in the format *(id, vector, metadata)*, where the *metadata* field is an optional extra field where we can store anything we want in a dictionary format. For this example, we will store the original text of the embeddings.

While uploading our data, we will batch everything to avoid pushing too much data in one go.

In [26]:
batch_size = 32

ids = [str(i) for i in range(shape[0])]
# create list of metadata dictionaries
meta = [{'text': text} for text in trec["train"]["text"][0]]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# let's view the index statistics
index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 7492}},
 'total_vector_count': 7492}

Perfect, we can see from `index.describe_index_stats` that we have a *4096-dimensionality* index populated with *7491* embeddings. The `indexFullness` metric tells us how full our index is, at the moment it is empty. Using the default value of one *p1* pod we can fit ~750K embeddings before the `indexFullness` reaches capacity. The [Usage Estimator](www.pinecone.io/pricing) can be used to identify the number of pods required for a given number of *n*-dimensional embeddings.

## Semantic Search

Now that we have our indexed vectors we can perform a few search queries. When searching we will first embed our query using Cohere, and then search using the returned vector in Pinecone.

In [27]:
query = "For cash-out refinances, can the new loan amount include the financing of real estate taxes?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='large',
    truncate='LEFT'
).embeddings

print(np.array(xq).shape)

# query, returning the top 10 most similar results
res = index.query(xq, top_k=10, include_metadata=True)
res

(1, 4096)


{'matches': [{'id': '2513',
              'metadata': {'text': 'include real estate taxes in the new loan '
                                   'amount as long as an escrow account is '
                                   'established,'},
              'score': 0.790164113,
              'values': []},
             {'id': '161',
              'metadata': {'text': '•    For cash-out transactions, the new '
                                   'loan amount cannot include the financing '
                                   'of real estate taxes'},
              'score': 0.77635622,
              'values': []},
             {'id': '2454',
              'metadata': {'text': 'for the initial purchase transaction to be '
                                   'a cash-out refinance.'},
              'score': 0.664563179,
              'values': []},
             {'id': '2449',
              'metadata': {'text': 'to applicable LTV restrictions. The '
                                   'mortgage amount f

The response from Pinecone includes our original text in the `metadata` field, let's print out the `top_k` most similar questions and their respective similarity scores.

In [28]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.79: include real estate taxes in the new loan amount as long as an escrow account is established,
0.78: •    For cash-out transactions, the new loan amount cannot include the financing of real estate taxes
0.66: for the initial purchase transaction to be a cash-out refinance.
0.65: to applicable LTV restrictions. The mortgage amount for cash-out refinance transactions may include
0.63: cash-out is sought, the refinance is subject to the same disclosures and closing
0.63: be reimbursed with the proceeds of the new mortgage loan
0.63: o    Cash proceeds from a cash-out refinance transaction on the subject
0.62: be underwritten as a Texas Home Equity cash-out refinance transaction. Even if no new
0.62: With the exception of real estate taxes that are more than 60 days delinquent, the borrower can
0.62: lien) will be treated as a cash-out refinance loan and the entirety of the blanket mortgage must be


Looks good, let's make it harder and replace *"depression"* with the incorrect term *"recession"*.

In [29]:
query = "Can I use a power of attorney for Cash-out transactions?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='large',
    truncate='LEFT'
).embeddings

# query, returning the top 10 most similar results
res = index.query(xq, top_k=10, include_metadata=True)

for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.67: In all other instances, power of attorney for closing documents with a loan is prohibited unless there is an
0.62: •    Power of Attorney may not be used on a Texas Home Equity loan
0.60: A Power of Attorney (POA) is a legal document giving one-person (described as the agent or attorney-in-
0.60: •    The Trust agreement authorizes the trustee to use a POA to delegate powers to an
0.60: •    Power of Attorney must be transaction specific unless it is a Military Durable POA (Freddie
0.59: POWER OF ATTORNEY
0.58: POWER OF ATTORNEY
0.57: For Freddie Mac transactions only, a POA is only allowable when the borrower’s
0.54: •    The agent under the POA is the borrower creating the Trust.
0.54: fact) power to legally bind the borrower enacting the POA to the mortgage transaction.


And again.

In [48]:
query = "Is a gifted fund for downpayment from a friend acceptable?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='large',
    truncate='LEFT'
).embeddings

# query, returning the top 10 most similar results
res = index.query(xq, top_k=10, include_metadata=True)

for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.68: that have subordinate financing, gift funds from an acceptable donor may be used to make the entire down
0.68: that have subordinate financing, gift funds from an acceptable donor may be used to make the entire down
0.68: that have subordinate financing, gift funds from an acceptable donor may be used to make the entire down
0.68: Gift funds may fund all or part of the down payment, closing costs, or financial
0.68: Gift funds may fund all or part of the down payment, closing costs, or financial
0.68: Gift funds may fund all or part of the down payment, closing costs, or financial
0.66: Gift funds:
0.66: Gift funds:
0.65: Gift funds can be used for down payment or to pay closing costs only. The following parameters apply:
0.65: Gift funds can be used for down payment or to pay closing costs. The following parameters apply:


Looks great, our semantic search pipeline is clearly able to identify the meaning between each of our queries and return the most semantically similar questions from the already indexed questions.

Once we're done with the index we delete it to save resources:

In [46]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.73: FOREIGN ASSETS
0.73: FOREIGN ASSETS
0.73: FOREIGN ASSETS
0.69: o   Foreign Assets
0.65: When the source of funds originates from assets located outside of the U.S. and its territories, those
0.65: When the source of funds originates from assets located outside of the U.S. and its territories, those
0.65: When the source of funds originates from assets located outside of the U.S. and its territories, those
0.65:                                          in a U.S or state regulated   foreign assets exchanged into
0.65:                                          in a U.S or state regulated   foreign assets exchanged into
0.64: â¢    Documented evidence of the foreign assets exchanged into U.S. dollars and held in a U.S. or state


In [52]:
pinecone.delete_index(index_name)

---